In [44]:
!pip install -q tensorflow scikit-learn seaborn matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os, cv2
from PIL import Image
import seaborn as sns
from sklearn.metrics import (top_k_accuracy_score, f1_score, roc_auc_score, auc)
from sklearn.preprocessing import label_binarize
import tensorflow as tf
from tensorflow.keras.models import load_model
from google.colab import drive

In [46]:
drive.mount('/content/drive', force_remount=True)
MODEL_PATH_1 = '/content/drive/MyDrive/FineTuned-EfficientNetB0_CNN_Model.keras'
MODEL_PATH_2 = '/content/drive/MyDrive/MobileNetV2_CNN_Model.keras'

Mounted at /content/drive


In [47]:
model_efficientnet = load_model(MODEL_PATH_1, safe_mode=False)
model_mobilenet = load_model(MODEL_PATH_2, safe_mode=False)
print("Models loaded!")

Models loaded!


In [48]:
DATA_FOLDER = '/content/drive/MyDrive/LandMark Images Pre-Processed'

**Data Loading**

In [17]:
# Load images
def load_images():

    splits = ['train', 'test', 'valid']
    X_train, y_train = [], []
    X_val, y_val = [], []
    X_test, y_test = [], []

    # Get landmark names (folders containing train/test/valid)
    classes = []
    for item in os.listdir(DATA_FOLDER):
        item_path = os.path.join(DATA_FOLDER, item)
        if os.path.isdir(item_path):
            subfolders = os.listdir(item_path)
            if all(split in subfolders for split in splits):
                classes.append(item)

    classes = sorted(classes)
    print(f"📁 Found {len(classes)} landmarks: {classes}")

    # Create class to index mapping
    class_to_idx = {name: i for i, name in enumerate(classes)}

    # Load images from each split
    for landmark in classes:
        for split in splits:
            # Direct path: landmark/train/ (no inner folder)
            source_path = os.path.join(DATA_FOLDER, landmark, split)

            if not os.path.exists(source_path):
                print(f"⚠️ Missing: {landmark}/{split}")
                continue

            # Get images directly from split folder
            images = [f for f in os.listdir(source_path)
                     if f.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))]

            print(f"📂 {landmark}/{split}: {len(images)} images")

            for img_name in images:
                try:
                    img_path = os.path.join(source_path, img_name)
                    img = np.array(Image.open(img_path).convert('RGB'))
                    img_resized = cv2.resize(img, (290, 290)) / 255.0

                    # Add to appropriate list
                    if split == 'train':
                        X_train.append(img_resized)
                        y_train.append(class_to_idx[landmark])
                    elif split == 'valid':
                        X_val.append(img_resized)
                        y_val.append(class_to_idx[landmark])
                    elif split == 'test':
                        X_test.append(img_resized)
                        y_test.append(class_to_idx[landmark])

                except Exception as e:
                    print(f"⚠️ Failed {img_name}: {e}")

    # Convert to numpy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_val = np.array(X_val)
    y_val = np.array(y_val)
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    print(f"\n✅ Loaded:")
    print(f"   Train: {len(X_train)} images")
    print(f"   Val:   {len(X_val)} images")
    print(f"   Test:  {len(X_test)} images")

    return X_train, X_val, X_test, y_train, y_val, y_test, classes

In [32]:
X_train, X_val, X_test, y_train, y_val, y_test, CLASS_NAMES = load_images()

📁 Found 24 landmarks: ['Adams Peak', 'Ancient City of Polonnaruwa', 'Beruwala Light House', 'British War Cemetery', 'Bundala National Park', 'Delft Island', 'Dowa Rock Temple', 'Ganagaramaya Temple', 'Henarathgoda Botanical Garden', 'Hortains Plain', 'Independance Square', 'Jaya Sri Maha Bodhi', 'Lotus Tower', 'Maligawa Buddha Statue', 'Nine Arches Bridge', 'Pinnawala Elephant Orphanage', 'Sigiriya', 'Sinharaja Forest', 'Sri Dalada Maligawa', 'Star Fort', 'Turtle Hatchery', 'Vavuniya Archaeological Museum', 'Wilapattu National Park', 'Yapahuwa Rock Fortress']
📂 Adams Peak/train: 18 images
📂 Adams Peak/test: 1 images
📂 Adams Peak/valid: 2 images
📂 Ancient City of Polonnaruwa/train: 18 images
📂 Ancient City of Polonnaruwa/test: 1 images
📂 Ancient City of Polonnaruwa/valid: 1 images
📂 Beruwala Light House/train: 18 images
📂 Beruwala Light House/test: 1 images
📂 Beruwala Light House/valid: 2 images
📂 British War Cemetery/train: 18 images
📂 British War Cemetery/test: 1 images
📂 British War 

**Models** **Evaluation**

In [71]:
# Function used to evaluate both cnn models
def evaluate_model(model, X_test, y_test, CLASS_NAMES, model_name):
    print(f"\n{'='*60}")
    print(f"EVALUATING: {model_name}")
    print(f"{'='*60}")

     # Single prediction with fixed seed
    tf.random.set_seed(42)
    y_pred_probs = model.predict(X_test, verbose=1, batch_size=32)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # 1. TOP-1 ACCURACY (standard accuracy)
    acc = accuracy_score(y_test, y_pred)
    print(f"\n1️⃣ ACCURACY:   {acc:.4f} ({acc*100:.2f}%)")

    # 2. TOP-3 ACCURACY (correct answer in top 3 guesses)
    top3_acc = top_k_accuracy_score(y_test, y_pred_probs, k=3)
    print(f"2️⃣ TOP-3 ACC:  {top3_acc:.4f} ({top3_acc*100:.2f}%)")

    # 3. F1-SCORE
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
    print(f"2️⃣ F1-SCORE:   {f1:.4f} ({f1*100:.2f}%)")

    # 4. ROC-AUC
    y_test_bin = label_binarize(y_test, classes=range(len(CLASS_NAMES)))
    roc_auc = roc_auc_score(y_test_bin, y_pred_probs, multi_class='ovr', average='macro')
    print(f"3️⃣ ROC-AUC:    {roc_auc:.4f} ({roc_auc*100:.2f}%)")

    return {
        'name': model_name,
        'accuracy': acc,
        'top3_accuracy': top3_acc,
        'f1_score': f1,
        'roc_auc': roc_auc
    }

In [72]:
# Evaluating the EfficientNetB0 model
results_eff = evaluate_model(model_efficientnet, X_test, y_test, CLASS_NAMES, "EfficientNetB0")


EVALUATING: EfficientNetB0
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step

1️⃣ ACCURACY:   0.7917 (79.17%)
2️⃣ TOP-3 ACC:  0.9583 (95.83%)
2️⃣ F1-SCORE:   0.7569 (75.69%)
3️⃣ ROC-AUC:    0.9710 (97.10%)


In [73]:
# Evaluating the MobileNetV2 model
results_mob = evaluate_model(model_mobilenet, X_test, y_test, CLASS_NAMES, "MobileNetV2")


EVALUATING: MobileNetV2
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

1️⃣ ACCURACY:   0.7083 (70.83%)
2️⃣ TOP-3 ACC:  0.8333 (83.33%)
2️⃣ F1-SCORE:   0.6389 (63.89%)
3️⃣ ROC-AUC:    0.9746 (97.46%)


In [75]:
print(f"\n{'='*60}")
print("                      FINAL COMPARISON")
print(f"{'='*60}")
print(f"{' Metric':<19} {'EfficientNetB0':<20} {'MobileNetV2':<18}")
print("-" * 60)
print(f"{'Accuracy':<22} {results_eff['accuracy']:<18.4f} {results_mob['accuracy']:<18.4f}")
print(f"{'Top-3 Accuracy':<22} {results_eff['top3_accuracy']:<18.4f} {results_mob['top3_accuracy']:<18.4f}")
print(f"{'F1-Score':<22} {results_eff['f1_score']:<18.4f} {results_mob['f1_score']:<18.4f}")
print(f"{'ROC-AUC':<22} {results_eff['roc_auc']:<18.4f} {results_mob['roc_auc']:<18.4f}")



                      FINAL COMPARISON
 Metric             EfficientNetB0       MobileNetV2       
------------------------------------------------------------
Accuracy               0.7917             0.7083            
Top-3 Accuracy         0.9583             0.8333            
F1-Score               0.7569             0.6389            
ROC-AUC                0.9710             0.9746            


In [79]:
best_model = results_eff if results_eff['f1_score'] > results_mob['f1_score'] else results_mob
print(f"\n   BEST MODEL: {best_model['name']}")
print(f"   Accuracy: {best_model['accuracy']:.4f}")
print(f"   Top-3 Accuracy: {best_model['top3_accuracy']:.4f}")
print(f"   F1-Score: {best_model['f1_score']:.4f}")
print(f"   ROC-AUC:  {best_model['roc_auc']:.4f}")


   BEST MODEL: EfficientNetB0
   Accuracy: 0.7917
   Top-3 Accuracy: 0.9583
   F1-Score: 0.7569
   ROC-AUC:  0.9710
